In [ ]:
from hi_sam.text_segmentation import load_sam_predictor, make_text_segmentation_args


arg = make_text_segmentation_args(
    model_type='sam_tss_l',
    checkpoint_path='sam_tss_l_textseg.pth',
    input_size=(512, 512),
)

sam = load_sam_predictor(arg)


In [ ]:
from diffusers.utils import load_image
from hi_sam.text_segmentation import run_text_stroke_segmentation


hf_dataset_base_url = "https://huggingface.co/datasets/GoGiants1/TMDBEval500/resolve/main/TMDBEval500/images/"
input_image = load_image(hf_dataset_base_url + '1.jpg')

mask_w_patchify = run_text_stroke_segmentation(sam_detector=sam, input_path=input_image, patch_mode=True)

In [ ]:
display(mask_w_patchify)

In [ ]:
mask_wo_patchify = run_text_stroke_segmentation(sam_detector=sam, input_path=input_image, patch_mode=False)

In [ ]:
display(mask_wo_patchify)